In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

import pdb

def translate_text(text, model, tokenizer, src_lang, tgt_lang):
    input_text = f"translate {src_lang} to {tgt_lang}: {text}"
    input_ids = tokenizer(input_text, return_tensors='pt').input_ids
    outputs = model.generate(input_ids, max_length=512)
    # pdb.set_trace()
    # print(outputs)
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(translated_text)
    return translated_text

model = T5ForConditionalGeneration.from_pretrained('./drive/MyDrive/AIProject/t5-small-translation')
tokenizer = T5Tokenizer.from_pretrained('./drive/MyDrive/AIProject/t5-small-translation')

text = "Hello how are you"
translated_text = translate_text(text, model, tokenizer, 'English', 'German')

print(translated_text)


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import pdb
import fitz


def pdf_to_text(file_path):
    """
    Convert a PDF file to text.

    Parameters:
    - file_path: Path to the PDF file.

    Returns:
    A string containing the text extracted from the PDF file.
    """
    text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

def translate_text(text, model, tokenizer, src_lang, tgt_lang):
    input_text = f"translate {src_lang} to {tgt_lang}: {text}"
    input_ids = tokenizer(input_text, return_tensors='pt').input_ids
    outputs = model.generate(input_ids, max_length=512)
    # pdb.set_trace()
    # print(outputs)
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(translated_text)
    return translated_text

model = T5ForConditionalGeneration.from_pretrained('./drive/MyDrive/AIProject/t5-small-translation')
tokenizer = T5Tokenizer.from_pretrained('./drive/MyDrive/AIProject/t5-small-translation')

text =  pdf_to_text("english.pdf")
translated_text = translate_text(text, model, tokenizer, 'English', 'German')

print(translated_text)


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/translate', methods=['POST'])
def translate():
    text = request.json.get('text')

    # Process text with your model (replace with your model logic)
    translated_text = your_translation_function(text)

    return jsonify({'translatedText': translated_text})

if __name__ == '__main__':
    app.run()


In [ ]:
from flask import Flask, request, jsonify
import os
from transformers import T5ForConditionalGeneration, T5Tokenizer
import fitz  # PyMuPDF

# Preload model and tokenizer
model_path = './drive/MyDrive/AIProject/t5-small-translation'
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

app = Flask(__name__)

def pdf_to_text(file_path):
    text = ""
    with fitz.open(file_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

def translate_text(text, src_lang='English', tgt_lang='German'):
    input_text = f"translate {src_lang} to {tgt_lang}: {text}"
    input_ids = tokenizer(input_text, return_tensors='pt').input_ids
    outputs = model.generate(input_ids, max_length=512)
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_text

@app.route('/translate_pdf', methods=['POST'])
def translate_pdf():
    # Check if a file is part of the request
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400
    file = request.files['file']
    # If the user does not select a file, the browser submits an
    # empty file without a filename.
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400
    if file:
        # Save file to a temporary path or process directly
        file_path = os.path.join('/tmp', file.filename)
        file.save(file_path)

        try:
            # Extract text from PDF
            text = pdf_to_text(file_path)

            # Translate text
            translated_text = translate_text(text)

            # Clean up: Remove the saved file after processing
            os.remove(file_path)

            return jsonify({'translatedText': translated_text})
        except Exception as e:
            # For production, you might want to log the error or send it to an error tracking service
            return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    app.run()


In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [ ]:
pip install PyMuPDF